### Nuestras tres hipótesis son:
-**La tasa de mortalidad de los hombres víctimas de ataques de tiburones es claramente mayor que la de las víctimas femeninas.**

-**No se puede establecer una relación clara entre la actividad practicada y las posibilidades de que el ataque sea mortal**

-**El momento del día influye claramente en la fatalidad del ataque**

En base a estas tres hipotesis realizaremos la limpieza y análisis del data set


In [2]:
import pandas as pd
import numpy as np
import re
import string
import src.cleaning as clean

In [3]:
# Importamos el archivo csv
df = pd.read_csv("../PROJECT-1/attacks.csv",encoding='latin1')
df

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Para empezar miramos que columnas tenemos en el data set.


In [4]:
# Para empezar miramos que columnas tenemos en el data set
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [5]:
# Ponemos los nombres en minuscula y en minuscula para facilitar el tratamiento
space = {i: i.rstrip() for i in df.columns}
df = df.rename(columns=space)
df.columns = df.columns.str.lower()

In [6]:
df.columns

Index(['case number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal (y/n)', 'time',
       'species', 'investigator or source', 'pdf', 'href formula', 'href',
       'case number.1', 'case number.2', 'original order', 'unnamed: 22',
       'unnamed: 23'],
      dtype='object')

In [7]:
# Eliminamos todas las columnas menos las que nos interesan
df = df.loc[:, ['time', 'sex','fatal (y/n)','activity']]
df

,time,sex,fatal (y/n),activity
0,18h00,F,N,Paddling
1,14h00 -15h00,F,N,Standing
2,07h45,M,N,Surfing
3,NaN,M,N,Surfing
4,NaN,M,N,Free diving
...,...,...,...,...
25718,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN


In [8]:
# Confirmamos que nos hemos quedado con las columnas que nos interesan
df.columns

Index(['time', 'sex', 'fatal (y/n)', 'activity'], dtype='object')

#### Ahora eliminamos todos los valores NaN para quedarnos con valores que nos sirvan en nuestro análisis.

In [9]:
# Consultamos cuantos NaN hay en cada columna 
df.isna().sum()

time           22775
sex            19986
fatal (y/n)    19960
activity       19965
dtype: int64

In [10]:
# Elimino las filas  NaN en el data set 
df = df[df['sex'].notnull() & df['fatal (y/n)'].notnull() & df['time'].notnull() & df['activity'].notnull()]

In [11]:
# Eliminamos valores duplicados en caso de que los hubiese
df.drop_duplicates(inplace=True)

/var/folders/5x/gn6s3css5x508rs__lb9ck2w0000gn/T/ipykernel_8423/1171080692.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [12]:
# Confirmamos que ya no quedan valores NaN
df.isna().sum()

time           0
sex            0
fatal (y/n)    0
activity       0
dtype: int64

#### Empezamos a tratar los valores que tenemos en el data set para que sean más cohesivos y nos ayuden en el análisis.

In [13]:
# En la columna de 'Sex' eliminamos todos los valores que no sean "M" o "F"
df = df[(df['sex'] == 'M') | (df['sex'] == 'F')]

In [14]:
# En la columna de 'Fatal (Y/N)' eliminamos todos los valores que no sean "Y" o "N"
df = df[(df['fatal (y/n)'] == 'Y') | (df['fatal (y/n)'] == 'N')]


In [15]:
# Filtramos los valores que no se ajusten al formato 00h00
horas = re.compile(r'^\d{1,2}h\d{2}$')
df = df[df['time'].str.match(horas)]

In [16]:
# Escribimos una funcion que categoriza las horas en la columna time y nos devuelve en que franja horaria ocurrieron los ataques
def time_day(time_str):
    if ':' in time_str:
        hour = int(time_str.split(':')[0])
        if 'PM' in time_str.upper() and hour != 12:
            hour += 12
    else:
        hour = int(time_str.split('h')[0])
#convertimos la hora en franja horaria
    if hour >= 5 and hour < 12:
        return 'Morning'
    elif hour >= 12 and hour < 17:
        return 'Afternoon'
    elif hour >= 17 and hour < 21:
        return 'Evening'
    else:
        return 'Night'

# Creamos una nueva columna con las franjas horarias 
df['time of the day'] = df['time'].apply(time_day)

# Eliminamos la columna de time
df.drop(['time'], axis=1, inplace= True)

In [17]:
def cohesive_activities(activity_str):    
    activity_str = activity_str.lower()
 # Defino un diccionario para mapear actividades similares a una actividad cohesiva
    activities = {
        'swim': 'Swimming',
        'Swimming': 'Swimming',
        "swim": 'Swimming',
        'surf': 'Surfing',
        'bath': 'Bathing',
        'div': 'Diving',
        'snork': 'Snorkeling',
        'padd': 'Paddle boarding',
        'play': 'Playing in the water',
        'wad': 'Wading',
        'body': 'Body boarding',
        'stand': 'Standing',
        'kite': 'Kiteboarding',
        'boog': 'Boogie boarding',
        'kayak': 'Kayaking',
        'spear': 'Spearfishing',
        "Fishing": 'Fishing',
        'net': 'Fishing'
    }
    for i, activity in activities.items():
        if re.search(i, activity_str):
            return activity
    return activity_str.capwords(activity_str)


#### Ahora comprobamos que los valores que han quedado en las columnas que nos interesan son los correctos.

In [18]:
df.value_counts('time of the day')

time of the day
Afternoon    617
Morning      460
Evening      223
Night         26
dtype: int64

In [19]:
df.value_counts('fatal (y/n)')

fatal (y/n)
N    1031
Y     295
dtype: int64

In [20]:
df.value_counts('sex')

sex
M    1098
F     228
dtype: int64

In [21]:
df['activity'].value_counts().head(20)

Surfing            173
Swimming           156
Wading              61
Spearfishing        59
Fishing             46
Standing            44
Body boarding       40
Snorkeling          35
Body surfing        28
Scuba diving        20
Swimming            19
Treading water      18
Kayaking            17
Boogie boarding     17
Diving              17
Bathing             16
Walking             13
Boogie Boarding     10
Free diving          9
Windsurfing          9
Name: activity, dtype: int64

#### Por último guardamos el pandas dataframe en un CSV para analizarlo.

In [22]:
df.to_csv('clean_data.csv', index=False)